<a href="https://colab.research.google.com/github/god7i11a/pynb/blob/main/CDC_hospital_wastewater.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Hospitalization and Wastewater Data Retrieval and Visualization
#         People's CDC   http://peoplescdc.org
#  https://covid.cdc.gov/covid-data-tracker/#hospitalizations-nhsn
#  https://covid.cdc.gov/covid-data-tracker/#wastewater-surveillance
#  https://www.cdc.gov/nwss/about-data.html#data-method
#               Updated 07/01/2024 @11:14 AM  by Les Schaffer

## Setup: be sure to run these first two cells

In [ ]:
import os
import re
import sys

try:
    from google.colab import userdata, output
    output.enable_custom_widget_manager()
    #api_key_id = userdata.get('api_key_id')
    #api_key_secret = userdata.get('api_key_secret')
    #api_app_token = userdata.get('api_app_token')
    api_key_id = 'teew3e5qkr4k81v5cobf2mg'
    api_key_secret = '4jujbp3kbw8di5qll6jzk1aid29una9unem6jlpdeu0jvtlbd2'
    api_app_token = '0X9q9rn19UjpZ05B7yIENlGru'
    ! pip install sodapy
    ! pip install ipympl
    ! pip install mapclassify
    ! pip install contextily
    COLAB = True
except ModuleNotFoundError:
    sys.path.append(os.path.abspath('../'))
    from tokens import api_key_id, api_key_secret, api_app_token
    COLAB=False

get_ipython().run_line_magic('matplotlib', 'ipympl')

from locale import atof
import math
import ipywidgets as widgets
from numpy import datetime_as_string, inf
from geopandas import GeoDataFrame, points_from_xy
from folium import Element, TileLayer, Map, Circle, CircleMarker, __version__ as fol_version
import branca.colormap as cm
from folium.map import Popup, Tooltip
from sodapy import Socrata
from pandas import DataFrame, to_datetime, Series, concat, isna, options
from matplotlib import pyplot as plt
import matplotlib.dates as mdates
import seaborn as sbrn
import contextily as cx
import xyzservices.providers as xyz

# set up Socrata clients for data queries
# hospital data by facility and by state
URL = 'healthdata.gov'
hclient = Socrata(URL, api_app_token,  username=api_key_id,   password=api_key_secret, timeout=20 )

# wastewater data by facility
URL = 'data.cdc.gov'
wclient = Socrata(URL, api_app_token,  username=api_key_id,   password=api_key_secret, timeout=60 )

In [ ]:
def format_axis(ax=None):
    _num2date = mdates.num2date

    def formatter(x, y):
        the_date = _num2date(x).date().isoformat()
        return f'x={the_date}, y={y:.4g}'

    if ax is None:
        ax = plt.gca()
    ax.format_coord = formatter

def _get_lonlat(x):
    x = x['geocoded_hospital_address']
    if isinstance(x, type(1.00)):
        return x, x
    if isinstance(x, str):
        # POINT (-91.634487925 35.768331958)
        lon, lat = map(atof, x[7:-1].split(' '))
        return (lon, lat)
    return x['coordinates']

pcr_conc_smoothed = 'pcr_conc_lin'

# Wastewater code

## need to handle visualization maps by-facility
## need to develop code to bring up to county

## https://www.nytimes.com/2024/01/10/upshot/covid-pandemic-wave.html

## https://www.cdc.gov/nwss/about-data.html
## https://www.cdc.gov/nwss/reporting.html

## initialization: needed functions and build waste treatment plant database

In [ ]:
# some  functions for later use
# x='CDC_BIOBOT_ny_505_Treatment plant_raw wastewater'
#             wwtp_jurisdiction     wwtp_id     reporting_jurisdiction  sample_location     sample_location_specify     key_plot_id                                         county_names    county_fips     population_served
# 252219      California            2526            California          Treatment plant     NaN                         CDC_VERILY_ca_2526_Treatment plant_post grit r...   Ventura         06111             250000
# 252487      Texas                 2527            Texas               Treatment plant     635                         CDC_VERILY_tx_2527_Treatment plant_635_raw was...   Travis          48453             529541
# 252884      California            834             California          Treatment plant     633                         CDC_VERILY_ca_834_Treatment plant_633_post gri...   Riverside       06065              91980
regex = re.compile(r'_[a-z][a-z]_(\d+)_')

def get_id(row):
    x = row.key_plot_id
    y = regex.search(x)
    if y:
        shortened = x[0:y.end()-1]
    else:
        print(row)
        shortened = x.split(' ')[0]
        print('id w/o #: ', shortened)
    # NWSS_az_1212 1212
    if not isna(row.sample_location_specify):
        shortened = shortened + f'_{row.sample_location_specify}'
    elif x.find('post grit')>-1:
        shortened=shortened+'_post'
    return shortened

def get_col_order(df, key_column):
    col_order = df[key_column].unique().tolist()
    col_order.sort()
    return col_order

def update_wwtp_id(df):
    df['wwtp_id'] = df.apply(get_id, axis=1)
    return df

def pop_averaged_conc(x):
    y = x[pcr_conc_smoothed] * x['population_served']
    pop = x['population_served'].sum()
    return y.sum()/pop

def pop_averaged_percentile(x):
    y = x['percentile'] * x['population_served']
    pop = x['population_served'].sum()
    return y.sum()/pop

def fix_microbial(x):
    if x.normalization == 'microbial':
        return x.pcr_conc_lin * x.population_served
    else:
        return x.pcr_conc_lin

def fix_flow_population(x):
    if x.normalization == 'flow-population':
        return x.pcr_conc_lin / x.population_served
    else:
        return x.pcr_conc_lin

## do we need 7-day averages???

In [ ]:
fieldStr = 'wwtp_jurisdiction,wwtp_id,reporting_jurisdiction,sample_location,sample_location_specify,key_plot_id,county_names,county_fips,population_served'
query = f"""
SELECT
{fieldStr}
WHERE
date_end>'2023-07-01T00:00:00.000'
LIMIT
520000
"""
results = wclient.get("2ew6-ywp6", query=query)
res0=DataFrame.from_records(results)

print(res0['key_plot_id'].count())

wwtp=res0.groupby('key_plot_id').nth(0).reset_index()
wwtp = wwtp.astype({'population_served': int, 'sample_location_specify': 'Int64' })
wwtp = update_wwtp_id(wwtp)
wwtp.to_csv('wwtp_db.csv')
mults = wwtp.groupby('wwtp_id',group_keys=False).apply(lambda x: x if x.key_plot_id.count()>1 else None)
print(mults)
mults.to_csv('multiples.csv')
wwtp

## by facility. can get all facilities in a state by inputting two letter state abbrev or particular facility by state+wwtp_id+sample_location_specify if existing
### enter state abbreviation at prompt

In [ ]:
wstate = "NY"   # @param {type:"string", description: 'Choose a two letter abbrev for state of interest'}

## By concentration
## https://data.cdc.gov/Public-Health-Surveillance/NWSS-Public-SARS-CoV-2-Concentration-in-Wastewater/g653-rqe2/about_data
## pcr_conc_smoothed

In [ ]:
RSRC ='g653-rqe2'

query = f"""
SELECT
*
WHERE
date>='2020-01-03T00:00:00.000' AND key_plot_id LIKE '%\\_{wstate.lower()}\\_%'
ORDER BY
date ASC
LIMIT
120000
"""

print(query)
results = wclient.get(RSRC, query=query)
res1=DataFrame.from_records(results)
res1[pcr_conc_smoothed] =  res1[pcr_conc_smoothed].astype(float)
res1['date'] = to_datetime(res1['date'])
# res1.to_csv('or_waste_prelim.csv')
res1 = res1.join(wwtp.set_index('key_plot_id'), on='key_plot_id')
# options.mode.use_inf_as_na = True
# res1['wwtp_id'].loc[res1['wwtp_id'].isna()]='NWSS_or_2800'
w1_col_order=get_col_order(res1, 'wwtp_id')
# need to groupby:
res1['percentile_from_conc'] = res1.groupby('key_plot_id', group_keys=False).apply(lambda x: 100*x[pcr_conc_smoothed]/ x[pcr_conc_smoothed].max())
res1['pcr_conc_lin'] = res1.apply(fix_flow_population, axis=1)
res1['pcr_conc_lin'] = res1.apply(fix_microbial, axis=1)
res1.to_csv(f'{wstate}_wastewater.csv')
res1

# why negative numbers for pcr_conc_smoothed???
#     key_plot_id   date      pcr_conc_smoothed    normalization       wwtp_id
# 57          527 2021-10-15      -2.875224e+06  flow-population   NWSS_ny_527
# 58           52 2021-10-15       2.960513e+07  flow-population    NWSS_ny_52
# 59         2182 2021-10-15       5.964828e+05  flow-population  NWSS_ny_2182
# 60          426 2021-10-15       1.424451e+07  flow-population   NWSS_ny_426
# 61          515 2021-10-15      -1.071621e+07  flow-population   NWSS_ny_515
# 62          522 2021-10-15      -4.880599e+06  flow-population   NWSS_ny_522
# 63          524 2021-10-15      -6.443013e+06  flow-population   NWSS_ny_524
# 64          529 2021-10-15      -1.401466e+07  flow-population   NWSS_ny_529

## Some of the counties (WWS, for example), have a vastly different scalling!!!! beware!!!

In [ ]:
fig,ax = plt.subplots(1,1,figsize=(14,8))
res1.groupby('key_plot_id').apply(lambda x: ax.semilogy(x.date, x[pcr_conc_smoothed], marker='.', ms=1, ls='', linewidth=1,  label=x.wwtp_id.iloc[0]))
ax.set_ylabel(pcr_conc_smoothed)
# plt.legend(ncols=3)

fig,ax=plt.subplots(1,1,figsize=(14,8))
res1.groupby('key_plot_id').apply(lambda x: ax.plot(x.date, x[pcr_conc_smoothed]/x[pcr_conc_smoothed].max(), marker='.', ms=2, ls='', linewidth=0.5,  label=x.wwtp_id.iloc[0]))
ax.set_ylim(0.,1)
ax.set_ylabel(f'{pcr_conc_smoothed}, normalized')
ax.set_title(f'all {wstate} wastewater stations reporting')
# plt.legend(ncols=3)



## NB: microbial normalization numbers MUCH smaller. 

In [ ]:
graph1 = sbrn.FacetGrid(res1, col='wwtp_id', col_order=w1_col_order, sharex=True, sharey=False, col_wrap=7)
graph1.map(plt.plot_date, 'date', pcr_conc_smoothed, ms=1)
graph1.tick_params('x', rotation=90)
if False:
    pcr_conc_lin_max =  res1[pcr_conc_smoothed].max()
    graph1.set(ylim=[0.,pcr_conc_lin_max])
graph1.savefig(f'{wstate}-wastewater-1.png')

graph4 = sbrn.FacetGrid(res1, col='wwtp_id', col_order=w1_col_order, sharex=True, sharey=False, col_wrap=7)
graph4.map(plt.plot_date, 'date', 'percentile_from_conc', ms=1)
graph4.tick_params('x', rotation=90)
graph4.savefig(f'{wstate}-wastewater-2a.png')

In [ ]:
from numpy import array
scale=1.7
w = scale*array([776, 754])
h = scale*array([851,390])


display(Image("Screenshot 2024-02-02 at 13-54-19 Data on COVID-19 and Mpox Wastewater Monitoring Biobot Analytics.png", width=w[0], height=h[0]))
display(Image("Screenshot 2024-02-02 at 13-54-35 Data on COVID-19 and Mpox Wastewater Monitoring Biobot Analytics.png", width=w[1], height=h[1]))


## Metric data
### https://data.cdc.gov/Public-Health-Surveillance/NWSS-Public-SARS-CoV-2-Wastewater-Metric-Data/2ew6-ywp6/about_data



In [ ]:
fieldStr = 'key_plot_id,date_start,date_end,percentile,ptc_15d'
query = f"""
SELECT
{fieldStr}
WHERE
date_end>'2020-01-01T00:00:00.000' AND key_plot_id LIKE '%\\_{wstate.lower()}\\_%'
LIMIT
140000
"""
results = wclient.get("2ew6-ywp6", query=query)
if not results:
  print(query)
  raise ValueError('Empty results!!!')
res2=DataFrame.from_records(results)
res2 = res2.join(wwtp.set_index('key_plot_id'), on='key_plot_id')
res2['percentile'] =  res2['percentile'].astype(float)
res2 = res2.astype({'population_served': int, 'sample_location_specify': 'Int64' })
res2['date_end'] = to_datetime(res2['date_end'])
w2_col_order=get_col_order(res2, 'wwtp_id')
percentile_from_conc=res1.rename(columns={'date':'date_end'}, inplace=False).set_index( keys=['key_plot_id', 'date_end'], inplace=False)[['percentile_from_conc','pcr_conc_lin']]
res2=res2.join(percentile_from_conc, on=('key_plot_id','date_end'))
res2.to_csv(f'{wstate}_metric_data_wastewater.csv')
res2

## Percentile
### needs comparison with the pcr_conc_smoothed

### every graph should have at least one 100%. but not.
#### This metric shows whether SARS-CoV-2 virus levels at a site are currently higher or lower than past historical levels at the same site.
#### 0% means levels are the lowest they have been at the site; 100% means levels are the highest they have been at the site.
#### Public health officials watch for increasing levels of the virus in wastewater over time and use this data to help make public health decisions.

### but maybe some help here: Calculating the Wastewater Viral Activity Level:

    Data Normalization:
        The type of data normalization used is based on the data that is submitted by the site.
            If both flow-population and microbial normalization values are available, flow-population normalization is used.
        After normalization, all concentration data is log transformed.
    Baseline Calculation:
        For each combination of site, data submitter, PCR target, lab methods, and normalization method, a baseline is established. The “baseline” is the 10th percentile of the log-transformed and normalized concentration data within a specific time frame.
            For site and method combinations (as listed above) with over six months of data, baselines are re-calculated every six calendar months (January 1st and July 1st) using the past 12 months of data.
            For sites and method combinations with less than six months of data, baselines are computed weekly until reaching six months, after which they remain unchanged until the next January 1st or July 1st, at which time baselines are re-calculated.
        The standard deviation for each site and method combination is calculated using the same time frame as the baseline.
    Wastewater Viral Activity Level Calculation:
        The number of standard deviations that each log-transformed concentration value deviates from the baseline (positive if above, negative if below) is calculated.
        This value is then converted back to a linear scale to form the Wastewater Viral Activity Level for the site and method combination.
        The Wastewater Viral Activity Levels from a site are averaged by week for all figures.
    Aggregation for National, Regional, and State Levels:
        We calculate the median Wastewater Viral Activity Levels among sites at national, regional, and state levels, excluding data from site/method combinations with less than 6 weeks of data.

In [ ]:
graph2 = sbrn.FacetGrid(res2, col='wwtp_id', col_order=w2_col_order, sharex=True, sharey=False, col_wrap=7)
graph2.map(plt.plot_date, 'date_end', 'percentile', ms=1)
graph2.map(plt.plot_date, 'date_end', 'percentile_from_conc', color='r', ms=1)
graph2.set(ylim=[0.,100.])
graph2.tick_params('x', rotation=90)
graph2.savefig(f'{wstate}-wastewater-2b.png')

In [ ]:
fig,ax=plt.subplots(1,1,figsize=(14,8))
res2.groupby('key_plot_id').apply(lambda x: ax.plot_date(x.date_end, x['percentile']/x['percentile'].max(), '.', ms=2, linewidth=0.5,  label=x.wwtp_id.iloc[0]))
ax.set_ylim(0.,1)
ax.set_ylabel('percentile')
ax.set_title(f'all {wstate} wastewater stations that are reporting')
# plt.legend(ncols=3)
# add subplot # stations reporting over time

### PTC_15D whatever that is, it doesn't LOOK like much. maybe i goofed here???

### quoteth the Law(d):

#### The percent change in SARS-CoV-2 RNA levels over the 15-day interval defined by 'date_start' and 'date_end'.
#### Percent change is calculated as the modeled change over the interval, based on linear regression of log-transformed SARS-CoV-2 levels.
#### SARS-CoV-2 RNA levels are wastewater concentrations that have been normalized for wastewater composition.

In [ ]:
cutoff = float(input('enter upper end cutoff:  '))
res2['ptc_15d'] =  res2['ptc_15d'].astype(float)
ptc_cropped = res2[res2['ptc_15d']<cutoff]

graph3 = sbrn.FacetGrid(ptc_cropped, col='wwtp_id', col_order=w2_col_order, sharex=True, sharey=False, col_wrap=7)
graph3.map(plt.plot_date, 'date_end', 'ptc_15d', ms=1)
graph3.tick_params('x', rotation=90)
graph3.savefig(f'{wstate}-wastewater-3b.png')

plt.figure()
ptc_cropped['ptc_15d'].plot(kind='hist', bins=400, logx=False, logy=True)
plt.xlabel('% change 15 days')
plt.title(f'Histogram of ptc_15d wastewater metric (>{cutoff} excluded)')

### Let's now look at state and national, and eventually, county

### pcr_conc_smoothed comes in slowly, over a number of weeks

In [ ]:
RSRC ='g653-rqe2'

query = f"""
SELECT
key_plot_id,
date,
{pcr_conc_smoothed},
normalization
WHERE
date>='2020-01-01T00:00:00.000'
LIMIT
800000
"""

results = wclient.get(RSRC, query=query)
res3=DataFrame.from_records(results)
res3[pcr_conc_smoothed] =  res3[pcr_conc_smoothed].astype(float)
res3['date'] = to_datetime(res3['date'])
res3=res3[res3[pcr_conc_smoothed].notna()]
res3.sort_values('date', inplace=True)
res3

In [ ]:

x=res3.groupby('date')[pcr_conc_smoothed].count()
fig, ax = plt.subplots(1,1, figsize=(10,8))
ax.plot_date(x.index, x, ms=1)
ax.set_ylabel('# stations reporting')
format_axis(ax)
ax.tick_params('x', rotation=90)
fig.suptitle(f'Number wastewater stations reporting {pcr_conc_smoothed}')
plt.savefig('num_waste_stations_pcr_conc.png')

### join dataset to the facility database

In [ ]:
full = res3.join(wwtp.set_index('key_plot_id'), on='key_plot_id')
full.sort_values('date', inplace=True)
scales=full.groupby('key_plot_id').apply(lambda x: x[pcr_conc_smoothed].max()< 1.0)
scales.rename('isSmall', inplace=True)
full=full.set_index('key_plot_id').join(scales, how='outer')
full['pcr_conc_lin'] = full.apply(fix_flow_population, axis=1)
full['pcr_conc_lin'] = full.apply(fix_microbial, axis=1)
full

In [ ]:
full.dtypes

In [ ]:
tot_pop_nat=wwtp['population_served'].sum()
tot_pop_nat

In [ ]:
new=res3[res3['date']>='2023-11-01'].groupby('date').count()
new

In [ ]:
nation_pop_avg = full.groupby(['date']).apply(pop_averaged_conc)
nation_mean = full.groupby(['date']).apply(lambda x: x[pcr_conc_smoothed].mean())
fig,ax = plt.subplots(1,1, figsize=(10,8))
ax.plot_date(nation_pop_avg.index, nation_pop_avg, 'r', ms=1, label='pop. averaged')
ax.plot_date(nation_mean.index, nation_mean, 'b', ms=1, label='mean')
ax.set_ylabel(pcr_conc_smoothed)
ax.tick_params('x', rotation=90)
ax.set_title(f'National pop. average of {pcr_conc_smoothed}')
ax.set_ylim(bottom=0., top=12000)
format_axis(ax)
plt.legend()
plt.savefig('wastewater_by_nation_pop_average.png')

In [ ]:
from IPython.display import Image, display
display(Image("/content/Screenshot 2024-02-19 at 13-36-38 Data on COVID-19 and Mpox Wastewater Monitoring Biobot Analytics.png", width=1400, height=800))

In [ ]:
state_conc = full.groupby(['date', 'wwtp_jurisdiction']).apply(lambda x: x[pcr_conc_smoothed].mean()).to_frame(name='concentration average')
state_conc['concentration (pop. averaged)']= full.groupby(['date', 'wwtp_jurisdiction']).apply(pop_averaged_conc).to_frame(name='concentration (pop. averaged)')
state_conc.reset_index(inplace=True)
graph_conc_state = sbrn.FacetGrid(state_conc, col='wwtp_jurisdiction', col_order=get_col_order(state_conc, 'wwtp_jurisdiction'), sharex=True, sharey=False, col_wrap=8)
graph_conc_state.map(plt.plot_date, 'date', 'concentration average', ms=1)
graph_conc_state.map(plt.semilogy, 'date', 'concentration (pop. averaged)', marker='.', color='r', ls='', ms=1)
graph_conc_state.tick_params('x', rotation=90)
graph_conc_state.savefig('wastewater_by_state.png')

In [ ]:
state_conc.to_csv('wastewater_pop_ave_by_state.csv')

## some of the pcr_conc_smooth data goes NEGATIVE!!!! what wid dat???

In [ ]:
negdf = res3[res3[pcr_conc_smoothed]<0].groupby('key_plot_id').apply(lambda x: x.pcr_conc_smoothed.count())
negdf.to_csv('below-zero.csv')
negdf

## now metric data

In [ ]:
RSRC ='2ew6-ywp6'

# date>='2020-01-03T00:00:00.000' AND key_plot_id LIKE '%_{wstate}_%'
# date='2023-12-22T00:00:00.000'
query = f"""
SELECT
key_plot_id,
date_end,
percentile
WHERE
date_end>='2020-01-01T00:00:00.000'
LIMIT
800000
"""

results = wclient.get(RSRC, query=query)
res4=DataFrame.from_records(results)
res4['percentile'] =  res4['percentile'].astype(float)
res4['date_end'] = to_datetime(res4['date_end'])
res4=res4[res4['percentile'].notna()]

x=res4.groupby('date_end')['percentile'].count()

full_per = res4.join(wwtp.set_index('key_plot_id'), on='key_plot_id')
full_per.sort_values('date_end')
full_per=full_per.replace(999., 0.)

## and plot em

In [ ]:
fig,ax = plt.subplots(1,1, figsize=(10,8))
ax.plot_date(x.index, x, ms=1)
ax.set_ylabel('# stations reporting')
format_axis(ax)
ax.tick_params('x', rotation=90)
fig.suptitle('Number wastewater stations reporting')
plt.savefig('num_waste_stations_per.png')

nation_percentile = full_per.groupby(['date_end']).apply(lambda x: x['percentile'].mean())
nation_percentile_pop_avg = full_per.groupby(['date_end']).apply(pop_averaged_percentile).to_frame(name='percentile (pop. averaged)')
fig,ax = plt.subplots(1,1, figsize=(10,8))
plt.plot_date(nation_percentile.index, nation_percentile, ms=1)
plt.plot_date(nation_percentile_pop_avg.index, nation_percentile_pop_avg, 'r.', ms=1)
plt.ylabel('percentile average')
format_axis(ax)
ax.tick_params('x', rotation=90)
plt.savefig('wastewater_by_nation_per.png')

state_percentile = full_per.groupby(['date_end', 'wwtp_jurisdiction']).apply(lambda x: x['percentile'].mean()).reset_index(name='percentile average')
graph_perc_state = sbrn.FacetGrid(state_percentile, col='wwtp_jurisdiction', col_order=get_col_order(state_percentile, 'wwtp_jurisdiction'), sharex=True, sharey=False, col_wrap=8)
graph_perc_state.map(plt.plot_date, 'date_end', 'percentile average', ms=1)
graph_perc_state.tick_params('x', rotation=90)
graph_perc_state.savefig('wastewater_by_state_per.png')

## Ok, who's the wiseguy who is entering 999.0 for percentile and when were they doing it???  Many many stations filled in data over a long period

In [ ]:
def _add_one(limit):
    num=0
    while num<limit:
        yield num
        num = num+1

df = res4[res4['percentile']==999.].set_index('key_plot_id', drop=True)
s = df['percentile'].groupby('key_plot_id').apply(lambda x: x.count())
s.rename('count', inplace=True)
df=df.join(s, how='outer')
df.sort_values('count', ascending=False, inplace=True)
add_one = _add_one(s.size)
fig,ax=plt.subplots(1,1, figsize=(8,8))
df.groupby('key_plot_id', sort=False).apply(lambda x: plt.plot_date(x['date_end'], 0.*x['percentile']+next(add_one), ms=1))
ax.tick_params('x', rotation=90)
ax.set_ylabel('station N')
ax.set_title('when station N reported 999.0 for percentile')
format_axis(ax)
df

# Lets collect case data for when correlations are desired with wastewater and hospitalizations

## https://data.cdc.gov/resource/yviw-z6j5.json

In [ ]:
RSRC = 'yviw-z6j5'
query = """
SELECT
fips_code, state, county, date, new_cases, new_deaths
WHERE
date>'2020-01-01T00:00:00.000'
ORDER BY
date
LIMIT
600000
"""

results = wclient.get(RSRC, query=query)
cases=DataFrame.from_records(results)
cases['date']=to_datetime(cases['date'])
cases=cases.astype({'new_cases': float})
cases=cases.astype({'new_deaths': float})
cases

## case data, pick a state

In [ ]:
cstate = "NY" # @param {type:"string", description: 'Choose a two letter abbrev for state of interest'}

cstate_cases = cases[cases['state']==cstate]

hs_col_order = cstate_cases['county'].unique().tolist()
hs_col_order.sort()

graph1 = sbrn.FacetGrid(cstate_cases, col='county', col_order=hs_col_order, sharex=True, sharey=False, col_wrap=7)
graph1.map(plt.plot_date, 'date', 'new_cases', ms=1)
graph1.set_ylabels('news cases per week')
graph1.tick_params('x', rotation=90)
graph1.savefig(f'{cstate}_cases.png')

graph1a = sbrn.FacetGrid(cstate_cases, col='county', col_order=hs_col_order, sharex=True, sharey=False, col_wrap=7)
graph1a.map(plt.plot_date, 'date', 'new_deaths', ms=1)
graph1a.set_ylabels('news deaths per week')
graph1a.tick_params('x', rotation=90)
graph1a.savefig(f'{cstate}_deaths.png')



## case data by state and nation

In [ ]:
stcases = cases.groupby(['state','date']).apply(lambda x: x.new_cases.sum()).reset_index(name='new_cases')
stdeaths= cases.groupby(['state','date']).apply(lambda x: x.new_deaths.sum()).reset_index(name='new_deaths')
print(stcases)
print(stdeaths)
hs_col_order = stcases['state'].unique().tolist()
hs_col_order.sort()

graph1 = sbrn.FacetGrid(stcases, col='state', col_order=hs_col_order, sharex=True, sharey=False, col_wrap=7)
graph1.map(plt.plot_date, 'date', 'new_cases', ms=1)
graph1.set_ylabels('news cases per week')
graph1.tick_params('x', rotation=90)
graph1.savefig('state_cases.png')

graph1 = sbrn.FacetGrid(stdeaths, col='state', col_order=hs_col_order, sharex=True, sharey=False, col_wrap=7)
graph1.map(plt.plot_date, 'date', 'new_deaths', ms=1)
graph1.set_ylabels('news deaths per week')
graph1.tick_params('x', rotation=90)
graph1.savefig('state_deaths.png')


fig,(ax1,ax2) = plt.subplots(2,1, figsize=(15,10))
ncases=stcases.groupby('date').apply(lambda x: x.new_cases.sum()).reset_index(name='new_cases')
ndeaths=stdeaths.groupby('date').apply(lambda x: x.new_deaths.sum()).reset_index(name='new_deaths')
print(ncases)
print(ndeaths)
ax1.plot_date(ncases.date, ncases.new_cases, '.')
ax1.set_ylabel('cases per week')
format_axis(ax1)
ax2.plot_date(ndeaths.date, ndeaths.new_deaths, '.')
ax2.set_ylabel('deaths per week')
format_axis(ax2)
fig.savefig('nation_cases_deaths.png')

# Hospitalization -- by facility
## https://healthdata.gov/Hospital/COVID-19-Reported-Patient-Impact-and-Hospital-Capa/anag-cw7u

## metadata

In [ ]:
RSRC ='anag-cw7u'
metadata = hclient.get_metadata(RSRC)
print([x['name'] for x in metadata['columns']])
print(metadata.keys())
print(metadata['rowsUpdatedBy'])
print(metadata['owner'])
metadata['metadata']
metadata['columns']

meta_amount = [x for x in metadata['columns'] if x['name'] == 'inpatient_beds_used_covid_7_day_avg']
meta_amount

## what dates are available

In [ ]:
RSRC ='anag-cw7u'
query="""
SELECT
distinct collection_week
ORDER BY
collection_week
LIMIT
3000
"""
# Use the 'where' argument to filter the data before downloading it
results = hclient.get(RSRC, query=query)   # AND hospital_subtype = 'Children's Hospitals'")
results=DataFrame.from_records(results)
print("Number of results downloaded: {}".format(len(results)))

results['collection_week'] = to_datetime(results['collection_week'])
ldates = results['collection_week'].sort_values().unique()
#ldates = list(map(lambda x: datetime_as_string(x, unit='D'), ldates))
ldates = datetime_as_string(ldates, unit='D')
last_date = ldates[-1]
print(ldates)
print(last_date)

## maps by facility, select date from list above by calendar/selector on right
## NOT too useful before 08-02-2020. took a few months to get hospital data up and running

In [ ]:
w = widgets.Dropdown(
    options= ldates,
    value=ldates[-1],
    description='Date:',
)

display(w)

## Note: if all_adult_hospital_inpatient_beds_7_day_avg is zero or NaN, we attempt to use all_adult_hospital_beds_7_day_avg

In [ ]:
RSRC ='anag-cw7u'
query = f"""
SELECT
state,
city,
hospital_name,
all_adult_hospital_beds_7_day_avg,
all_adult_hospital_inpatient_beds_7_day_avg,
total_adult_patients_hospitalized_confirmed_covid_7_day_avg,
total_adult_patients_hospitalized_confirmed_and_suspected_covid_7_day_avg,
collection_week,
hospital_subtype,
fips_code,
hospital_pk,
is_metro_micro,
geocoded_hospital_address
WHERE
collection_week=='{w.value}T00:00:00.000000000'
AND hospital_subtype != "Childrens Hospitals"
ORDER BY
collection_week
LIMIT
40000
"""

# Use the 'where' argument to filter the data before downloading it
results = hclient.get(RSRC, query=query)   # AND hospital_subtype = 'Children's Hospitals'")
resfac=DataFrame.from_records(results)
print("Number of results downloaded: {}".format(len(results)))
resfac=resfac.replace('-999999.0', '2.0')
resfac['total_adult_patients_hospitalized_confirmed_covid_7_day_avg']=resfac['total_adult_patients_hospitalized_confirmed_covid_7_day_avg'].astype(float)
resfac['all_adult_hospital_inpatient_beds_7_day_avg']=resfac['all_adult_hospital_inpatient_beds_7_day_avg'].astype(float)
resfac['all_adult_hospital_beds_7_day_avg']=resfac['all_adult_hospital_beds_7_day_avg'].astype(float)
resfac['collection_week'] = to_datetime(resfac['collection_week'])
resfac.fillna(0., inplace=True)
resfac['frac'] = resfac['total_adult_patients_hospitalized_confirmed_covid_7_day_avg']/resfac['all_adult_hospital_inpatient_beds_7_day_avg']
nobeds = (resfac['all_adult_hospital_inpatient_beds_7_day_avg'] == 0.) & (resfac['total_adult_patients_hospitalized_confirmed_covid_7_day_avg'] !=0.)
resfac.loc[nobeds,'frac'] = resfac.loc[nobeds,'total_adult_patients_hospitalized_confirmed_covid_7_day_avg'] / resfac.loc[nobeds,'all_adult_hospital_beds_7_day_avg']
resfac.replace(inf, 1.0, inplace=True)

resfac[['lon','lat']]=resfac.apply(_get_lonlat, axis=1, result_type='expand')
geo = points_from_xy(resfac['lon'], resfac['lat'])
resfac = GeoDataFrame(resfac, geometry=geo)
resfac = resfac.set_crs(epsg=4236, allow_override=True)
resfac = resfac.to_crs("ESRI:102003")
resfac[nobeds]

## Histogram of hospitals in each category of "fraction of beds used for covid"

### discrepancies here for 12/26/21   
### https://data.rgj.com/covid-19-hospital-capacity/facility/hood-memorial-hospital/191309/
### hospital_pk = 050295

In [ ]:
# fracmin, bedsmin
#   (0.02, 30.0)
fracmin = 0.01 # @param {type:"number"}
bedsmin = 25 # @param {type:"integer"}

included = (resfac['frac']>=fracmin) & (resfac['all_adult_hospital_inpatient_beds_7_day_avg']>=bedsmin)
excluded = ~ included

mapfac=resfac.loc[included]
fracmax = mapfac['frac'].max()
mapfac = mapfac.drop(columns='collection_week')
mapfac['percent'] = mapfac['frac'].apply(lambda x: f'{100.*x:.2f}%')
mapfac.rename(columns={'all_adult_hospital_inpatient_beds_7_day_avg':'beds'}, inplace=True)

hospital_filtered_out = resfac[excluded]
hospital_filtered_out = hospital_filtered_out.drop(columns='collection_week')
hospital_filtered_out['percent'] = hospital_filtered_out['frac'].apply(lambda x: f'{100.*x:.2f}%')
hospital_filtered_out.rename(columns={'all_adult_hospital_inpatient_beds_7_day_avg':'beds'}, inplace=True)
hospital_filtered_out.fillna(0., inplace=True)
hospital_filtered_out = hospital_filtered_out[hospital_filtered_out['total_adult_patients_hospitalized_confirmed_covid_7_day_avg']!=0.]

nonzero = resfac['frac']!=0.
fracdata = resfac.loc[nonzero]
print(fracdata['frac'].describe())
print(mapfac['frac'].describe())
print(hospital_filtered_out['frac'].describe())

cmap_color_list = ['yellow', 'orange', 'red', 'green', 'blue']

qbars = [.25,.5,.75, 1]
q1=fracdata['frac'].quantile(qbars)
q2=mapfac['frac'].quantile(qbars)
q3=hospital_filtered_out['frac'].quantile(qbars)

fracmax = min(0.75, mapfac['frac'].max())
cindex_mapfac = [fracmin, q2.loc[.25], q2.loc[.5], q2.loc[.75], fracmax]
print('mapfac cindex = ', cindex_mapfac)
fracmax = min(0.75, hospital_filtered_out['frac'].max())
cindex_excluded = [0., q3.loc[.25], q3.loc[.5], q3.loc[.75], fracmax]

USELOG=True

fig,(ax1,ax2, ax3)= plt.subplots(3,1, figsize=(10,12), sharex=True)
_, _bins, _= ax1.hist(fracdata.frac, bins=200, range=[0,1.0], log=USELOG)
ax1.set_ylabel('# of hospitals')
ax1.set_title('full dataset')
ax1.grid()
#format_axis(ax1)
ax2.hist(mapfac['frac'], bins=_bins, log=USELOG)
ax2.set_title('selected dataset')
ax2.set_ylabel('# of hospitals')
ax2.grid()
#format_axis(ax2)

ax3.hist(hospital_filtered_out['frac'], bins=_bins, log=USELOG)
ax3.set_title('excluded dataset')
ax3.set_xlabel('fraction of inpatient beds used for COVID patients')
ax3.set_ylabel('# of hospitals')
ax3.grid()
#format_axis(ax3)

quartile_linetype = '-.'
q1.apply(lambda x: ax1.plot([x,x],[1,100], quartile_linetype))
q2.apply(lambda x: ax2.plot([x,x],[1,100], quartile_linetype))
q3.apply(lambda x: ax3.plot([x,x],[1,100], quartile_linetype))

fig1,ax3 = plt.subplots(1,1, figsize=(10,10))
ax3.plot(mapfac['beds'], mapfac['total_adult_patients_hospitalized_confirmed_covid_7_day_avg'], 'b.', ms=2, label='selected data')
ax3.plot(hospital_filtered_out['beds'], hospital_filtered_out['total_adult_patients_hospitalized_confirmed_covid_7_day_avg'], 'r.', ms=2, label = 'excluded data')
ax3.set_xlabel('# inpatient beds')
ax3.set_ylabel('total_adult_patients_hospitalized_confirmed_covid_7_day_avg')
ax3.legend()


## Zoom in and pan, then hover or click on circles
## circle size proportional to # of beds in the next three maps

In [ ]:
# us_discrete_bubble_bounds = dict(west=-126, east=-67, south=24.9, north=49.2)
# folium version = 0.14
title=f'# beds >= {bedsmin} and frac >= {fracmin} for {w.value}'
print(title)
print(cindex_mapfac)

m=Map(location=[44,-95], width=2300, height=900, min_lat=24.9, max_lat=49.2, min_lon=-126, max_lon=-67, max_bounds=False, zoom_start=5)

#TileLayer("CartoDB positron", show=False).add_to(m)
title_html = f'<h1 align="center", style="font-size:20px">{title}</h1>'
m.get_root().html.add_child(Element(title_html))

# https://python-visualization.github.io/folium/latest/advanced_guide/colormaps.html
# need to make index a function of the histogram above!!! OR, use method quantiles!!!
colormap = cm.LinearColormap(cmap_color_list,index=cindex_mapfac, vmin=cindex_mapfac[0], vmax=cindex_mapfac[-1], tick_labels=cindex_mapfac, caption='fraction beds used for covid')

#     cmarkD = {Circle: 20000.0*local_deformation, CircleMarker:100 }
for hosp in mapfac.itertuples():
    local_deformation = math.cos(hosp.lat * math.pi / 180)

    CircleMarker(
        location=[hosp.lat, hosp.lon],
        popup=Popup('<b>%s<br>  %s<br>  %.1f beds' % (hosp.hospital_name, hosp.percent, hosp.beds), max_width=300),
        radius=int(hosp.beds * 0.02)+10,
        tooltip='%s'%hosp.percent,
        color=colormap(hosp.frac),
        fill=True,
        fill_color=colormap(hosp.frac)
    ).add_to(m)
m.add_child(colormap)
m.save(f'facility_bed_percent_{w.value}.html')
m

## which facilities not reporting ANY hospitalizations!?!?!?!?!?

In [ ]:
no_hosp = resfac[resfac['total_adult_patients_hospitalized_confirmed_covid_7_day_avg']==0 | resfac['total_adult_patients_hospitalized_confirmed_covid_7_day_avg'].isna()]
cnt = no_hosp['total_adult_patients_hospitalized_confirmed_covid_7_day_avg'].count()
no_hosp.rename(columns={'all_adult_hospital_inpatient_beds_7_day_avg':'beds'}, inplace=True)
m=Map(location=[44,-95], width=2300, height=900, min_lat=24.9, max_lat=49.2, min_lon=-126, max_lon=-67, max_bouds=True, zoom_start=5)

#TileLayer("CartoDB positron", show=False).add_to(m)
title=f'{cnt} Hospitals reporting NO hospitalizations on {w.value} (circle size proportional to # beds)'
title_html = f'<h1 align="center", style="font-size:20px">{title}</h1>'
m.get_root().html.add_child(Element(title_html))

#     cmarkD = {Circle: 20000.0*local_deformation, CircleMarker:100 }
for hosp in no_hosp.itertuples():
    local_deformation = math.cos(hosp.lat * math.pi / 180)

    CircleMarker(
        location=[hosp.lat, hosp.lon],
        tooltip=Tooltip('<b>%s<br>  %.1f beds' % (hosp.hospital_name, hosp.beds)),
        radius=int(hosp.beds * .05)+1,
        fill=True,
    ).add_to(m)
m.save('facility_no_hosp.html')
m

## what's excluded from the maps, small hospitals with few non-zero covid cases
## Note that the large circles usually have a couple patients and a couple to a hadful or two of beds.
## If a hospital reports a few covid patients and 0.0 beds, we assume they found just enough beds and assign 100%
## click on a circle for details

In [ ]:
# us_discrete_bubble_bounds = dict(west=-126, east=-67, south=24.9, north=49.2)
# folium version = 0.14
title=f'filtered out hospitals'
print(title)

m=Map(location=[44,-95], width=2300, height=900, min_lat=24.9, max_lat=49.2, min_lon=-126, max_lon=-67, max_bouds=True, zoom_start=5)

#TileLayer("CartoDB positron", show=False).add_to(m)
title_html = f'<h1 align="center", style="font-size:20px">{title}</h1>'
m.get_root().html.add_child(Element(title_html))

# https://python-visualization.github.io/folium/latest/advanced_guide/colormaps.html
# need to make index a function of the histogram above!!!
colormap = cm.LinearColormap(['orange', 'red', 'green', 'blue', 'black'], vmin=cindex_excluded[0], vmax=cindex_excluded[-1], index=cindex_excluded)

#     cmarkD = {Circle: 20000.0*local_deformation, CircleMarker:100 }
for hosp in hospital_filtered_out.itertuples():
    local_deformation = math.cos(hosp.lat * math.pi / 180)

    CircleMarker(
        location=[hosp.lat, hosp.lon],
        popup=Popup('<b>%s<br>  %s<br>  %s covid patients<br> %.1f beds' % (hosp.hospital_name, hosp.percent, hosp.total_adult_patients_hospitalized_confirmed_covid_7_day_avg, hosp.beds), max_width=300, index=cindex_excluded),
        radius=int(hosp.beds * .05)+1,
        tooltip='%s'%hosp.percent,
        color=colormap(hosp.frac),
        fill=True,
        fill_color=colormap(hosp.frac)
    ).add_to(m)
m.add_child(colormap)
m.save(f'facility_bed_percent_{w.value}.html')
m

In [ ]:
hospital_filtered_out

## a Louisiana hospital with reporting oddities

In [ ]:
RSRC ='anag-cw7u'
query = f"""
SELECT
state,
city,
hospital_name,
all_adult_hospital_beds_7_day_avg,
all_adult_hospital_inpatient_beds_7_day_avg,
total_adult_patients_hospitalized_confirmed_covid_7_day_avg,
total_adult_patients_hospitalized_confirmed_and_suspected_covid_7_day_avg,
collection_week,
hospital_subtype,
fips_code,
hospital_pk,
is_metro_micro,
geocoded_hospital_address
WHERE
hospital_pk=='191309'
ORDER BY
collection_week
LIMIT
40000
"""

results = hclient.get(RSRC, query=query)   # AND hospital_subtype = 'Children's Hospitals'")
resfac=DataFrame.from_records(results)
print("Number of results downloaded: {}".format(len(results)))
resfac=resfac.replace('-999999.0', '2.0')
resfac['total_adult_patients_hospitalized_confirmed_covid_7_day_avg']=resfac['total_adult_patients_hospitalized_confirmed_covid_7_day_avg'].astype(float)
resfac['all_adult_hospital_inpatient_beds_7_day_avg']=resfac['all_adult_hospital_inpatient_beds_7_day_avg'].astype(float)
resfac['all_adult_hospital_beds_7_day_avg']=resfac['all_adult_hospital_beds_7_day_avg'].astype(float)
resfac['collection_week'] = to_datetime(resfac['collection_week'])
resfac['frac'] = resfac['total_adult_patients_hospitalized_confirmed_covid_7_day_avg']/resfac['all_adult_hospital_inpatient_beds_7_day_avg']
nobeds = (resfac['all_adult_hospital_inpatient_beds_7_day_avg'] == 0.) & (resfac['total_adult_patients_hospitalized_confirmed_covid_7_day_avg'] !=0.)
resfac.loc[nobeds,'frac'] = resfac.loc[nobeds,'total_adult_patients_hospitalized_confirmed_covid_7_day_avg'] / resfac.loc[nobeds,'all_adult_hospital_beds_7_day_avg']
resfac.fillna(0., inplace=True)
resfac.replace(inf, 1.0, inplace=True)
fig,ax=plt.subplots(1,1,figsize=(10,10))
ax.plot_date(resfac['collection_week'], resfac['frac'], ms=2)
ax.set_ylabel('bed fraction for covid in LA Amite hospital')
ax.tick_params('x', rotation=90)
ax.set_title('Spock! explain!!!')
fig.savefig('amite_la_hospital.png')
format_axis(ax)
resfac

In [ ]:
# https://www.martinalarcon.org/2018-12-31-d-geopandas/
# https://docs.stadiamaps.com/guides/migrating-from-stamen-map-tiles/

ax = mapfac.plot('frac', scheme='quantiles', k=5, categorical=False, figsize=(10, 10), legend=True)
ax.set_xlim(-.24444e7, 0.2274e7)
ax.set_ylim(-1.399e6, 1.55e6)
api_key = "c9a3c30d-586e-4eca-9163-49c747840f30"
URL = 'https://tiles.stadiamaps.com/tiles/stamen_terrain/{z}/{x}/{y}{r}.png?api_key=c9a3c30d-586e-4eca-9163-49c747840f30'
cx.add_basemap(ax, crs=mapfac.crs, source=URL)
ax.tick_params(left=False, right=False, labelleft=False, labelbottom=False, bottom=False)
ax.set_title(f'Fraction of covid hospital beds ({ldates[-1]})')
ax.get_legend().set_title('Quantiles')

## make tables and plots
### Children's hospitals: covid bed fraction > 20%

In [ ]:
query = """
SELECT
collection_week,
hospital_name,
all_pediatric_inpatient_beds_7_day_avg,
total_pediatric_patients_hospitalized_confirmed_and_suspected_covid_7_day_avg,
inpatient_beds_7_day_avg
WHERE
collection_week>'2020-03-01T00:00:00.000'
AND hospital_subtype = "Childrens Hospitals"
ORDER BY
collection_week
LIMIT
40000
"""
RSRC ='anag-cw7u'
results = hclient.get(RSRC, query=query)
res=DataFrame.from_records(results)
res=res.replace('-999999.0', '2.0')
res['total_pediatric_patients_hospitalized_confirmed_and_suspected_covid_7_day_avg'] = res['total_pediatric_patients_hospitalized_confirmed_and_suspected_covid_7_day_avg'].astype(float)
res['all_pediatric_inpatient_beds_7_day_avg']=res['all_pediatric_inpatient_beds_7_day_avg'].astype(float)
res['inpatient_beds_7_day_avg'] = res['inpatient_beds_7_day_avg'].astype(float)
res['percent'] = 100*res['total_pediatric_patients_hospitalized_confirmed_and_suspected_covid_7_day_avg']/res['all_pediatric_inpatient_beds_7_day_avg']
# cases where all_pediatric_inpatient_beds_7_day_avg is reported as 0!!!! then use inpatient_beds_7_day_avg
use_inpatient_beds = res['all_pediatric_inpatient_beds_7_day_avg']==0.0
res.loc[use_inpatient_beds, 'percent'] = 100*res[use_inpatient_beds]['total_pediatric_patients_hospitalized_confirmed_and_suspected_covid_7_day_avg']/res[use_inpatient_beds]['inpatient_beds_7_day_avg']

num = res.loc[:,'percent'].count()
print(f'Number of times facilities reporting = {num}')
hidden = res['total_pediatric_patients_hospitalized_confirmed_and_suspected_covid_7_day_avg']==2.0
num = res.loc[hidden,'percent'].count()
print(f'Number of times facilities reporting -999999 --> 2 = {num}')
plt.figure()
res.loc[~hidden, 'percent'].plot(kind='hist', bins=100, logy=True)
plt.gcf().suptitle('# reports of percent usage')
plt.gca().set_xlabel('percent usage')
res[res['percent']>=20].sort_values('percent', ascending=False)

### lets look at average usage over the entire pandemic > 5% !!!

In [ ]:
df=res.groupby(['hospital_name',])['percent'].mean()
df[df>=5.]

### Pediatric admission, nationwide

In [ ]:
query = """
SELECT
previous_day_admission_pediatric_covid_confirmed_7_day_sum,
collection_week
WHERE
collection_week>'2020-01-03T00:00:00.000' AND hospital_subtype = "Childrens Hospitals"
ORDER BY
collection_week ASC
LIMIT
30000
"""
RSRC ='anag-cw7u'
results = hclient.get(RSRC, query=query)
res=DataFrame.from_records(results)
res=res.replace('-999999', '2')
res['collection_week'] = to_datetime(res['collection_week'])
res=res[res['previous_day_admission_pediatric_covid_confirmed_7_day_sum'].notna()]
res['previous_day_admission_pediatric_covid_confirmed_7_day_sum']=res['previous_day_admission_pediatric_covid_confirmed_7_day_sum'].astype(int)
df=res.groupby('collection_week').sum()
plt.figure()
plt.plot(df['previous_day_admission_pediatric_covid_confirmed_7_day_sum'])
plt.tick_params('x', rotation=90)
plt.title('pediatric admissions -- 7-day sums')
format_axis()
res

## Adults

In [ ]:
query = """
SELECT
previous_day_admission_adult_covid_confirmed_7_day_sum,
total_adult_patients_hospitalized_confirmed_covid_7_day_avg,
total_adult_patients_hospitalized_confirmed_covid_7_day_sum,
collection_week
WHERE
collection_week>'2020-01-03T00:00:00.000' AND hospital_subtype != "Childrens Hospitals"
ORDER BY
collection_week ASC
LIMIT
1000000
"""
RSRC ='anag-cw7u'
results = hclient.get(RSRC, query=query)
ares=DataFrame.from_records(results)
ares=ares.replace('-999999', '2')
ares=ares.replace('-999999.0', '2.0')
ares['collection_week'] = to_datetime(ares['collection_week'])

ares=ares[ares['previous_day_admission_adult_covid_confirmed_7_day_sum'].notna()]
ares['previous_day_admission_adult_covid_confirmed_7_day_sum']=ares['previous_day_admission_adult_covid_confirmed_7_day_sum'].astype(int)

ares=ares[ares['total_adult_patients_hospitalized_confirmed_covid_7_day_avg'].notna()]
ares['total_adult_patients_hospitalized_confirmed_covid_7_day_avg']=ares['total_adult_patients_hospitalized_confirmed_covid_7_day_avg'].astype(float)

ares=ares[ares['total_adult_patients_hospitalized_confirmed_covid_7_day_sum'].notna()]
ares['total_adult_patients_hospitalized_confirmed_covid_7_day_sum']=ares['total_adult_patients_hospitalized_confirmed_covid_7_day_sum'].astype(float)
ares

In [ ]:
df=ares.groupby('collection_week').sum()
fig, (ax1,ax2) = plt.subplots(2,1, sharex=True, figsize=(10,8))
ax1.plot(df['previous_day_admission_adult_covid_confirmed_7_day_sum']/7)
ax1.tick_params('x', rotation=90)
ax1.set_title('adult admissions -- 7-day ave')

ax2.plot(df['total_adult_patients_hospitalized_confirmed_covid_7_day_avg'])
ax2.tick_params('x', rotation=90)
ax2.set_title('adult hospitalized -- 7-day ave')

# by state and nation, adult
## metadata first, only if needed

In [ ]:
RSRC ='g62h-syeh'
metadata = hclient.get_metadata(RSRC)

# print([x['name'] for x in metadata['columns']])
print(metadata.keys())
print(metadata['rowsUpdatedBy'])
print(metadata['owner'])
metadata['metadata']
metadata['columns']

## get the data

In [ ]:
RSRC ='g62h-syeh'
query = """
SELECT
hospital_onset_covid,
deaths_covid,
previous_day_admission_adult_covid_confirmed,
percent_of_inpatients_with_covid,
state,
date
WHERE
date>'2020-01-03T00:00:00.000'
ORDER BY
date ASC
LIMIT
100000
"""
colL = ['hospital_onset_covid', 'deaths_covid', 'previous_day_admission_adult_covid_confirmed', 'percent_of_inpatients_with_covid']
results = hclient.get(RSRC, query=query)
hsres=DataFrame.from_records(results)
hsres.fillna(0,inplace=True)
hsres['date'] = to_datetime(hsres['date'])
for colN in colL[:-1]:
    hsres[colN] = hsres[colN].astype(int)

hsres[colL[-1]] = hsres[colL[-1]].astype(float)
hsres.sort_values(['state','date'], inplace=True, ignore_index=True)
hs_col_order = hsres['state'].unique().tolist()
hs_col_order.sort()

moving_average=True
if moving_average:
    col_avgL = [col+'_avg' for col in colL]
    grouped = hsres.groupby('state', group_keys=True)
    rolldf = grouped.rolling('7D', on='date', axis=0, center=False).mean()
    # rolldf.reset_index(drop=True)


hsres[col_avgL] = rolldf.reset_index()[colL]
print(f"last date: {hsres['date'].sort_values().unique()[-1]}")
hsres.to_csv('hsres.csv')

## Visualize the data sets
### Nationwide

In [ ]:
fig,axL = plt.subplots(4,1, sharex=True, figsize=(12,9))
fig.suptitle(f'Nationwide')
ylabL = ('#', '#', '#', 'fraction')
for stat, stat_avg, ax, ylab in zip( colL, col_avgL, axL, ylabL):
    national = hsres.groupby('date')[stat].sum()
    ax.plot_date(national.index,national,ms=1)
    national = hsres.groupby('date')[stat_avg].sum()
    ax.plot_date(national.index,national,'k-.', ms=1)
    ax.tick_params('x', rotation=90)
    ax.set_ylabel(ylab)
    ax.set_title(stat)
    format_axis(ax)

fig.savefig(f'US-adult_hosp_stats.png')

### hospital_onset_covid

In [ ]:
graph1 = sbrn.FacetGrid(hsres, col='state', col_order=hs_col_order, sharex=True, sharey=False, col_wrap=7)
graph1.map(plt.plot_date, 'date', 'hospital_onset_covid', ms=1)
graph1.map(plt.plot_date, 'date', 'hospital_onset_covid_avg', color='k', linestyle='-.', ms=1)
graph1.set_ylabels('hospital_onset_covid')
graph1.tick_params('x', rotation=90)
graph1.savefig('hospital_onset_covid.png')

### deaths_covid

In [ ]:
graph2 = sbrn.FacetGrid(hsres, col='state', col_order=hs_col_order, sharex=True, sharey=False, col_wrap=7)
graph2.map(plt.plot_date, 'date', 'deaths_covid', ms=1)
graph2.map(plt.plot_date, 'date', 'deaths_covid_avg', color='k', linestyle='-.', ms=1)
graph2.tick_params('x', rotation=90)
graph2.set_ylabels('deaths_covid')
graph2.savefig('deaths_covid.png')

### previous_day_admission_adult_covid_confirmed

In [ ]:
graph3 = sbrn.FacetGrid(hsres, col='state', col_order=hs_col_order, sharex=True, sharey=False, col_wrap=7)
graph3.map(plt.plot_date, 'date', 'previous_day_admission_adult_covid_confirmed', ms=1)
graph3.map(plt.plot_date, 'date', 'previous_day_admission_adult_covid_confirmed_avg', color='k', linestyle='-.', ms=1)
graph3.set_ylabels('admission_adult_covid_confirmed')
graph3.tick_params('x', rotation=90)
graph3.savefig('previous_day_admission_adult_covid_confirmed.png')

### percent_of_inpatients_with_covid

In [ ]:
graph4 = sbrn.FacetGrid(hsres, col='state', col_order=hs_col_order, sharex=True, sharey=False, col_wrap=7)
graph4.map(plt.plot_date, 'date', 'percent_of_inpatients_with_covid', ms=1)
graph4.map(plt.plot_date, 'date', 'percent_of_inpatients_with_covid_avg', color='k', linestyle='-.', ms=1)
graph4.set_ylabels('percent_of_inpatients_with_covid')
graph4.tick_params('x', rotation=90)
graph4.savefig('percent_of_inpatients_with_covid.png')

## data summary over pandemic, by state

In [ ]:
dataD = {}
totalD = {'state':'TOTAL/AVE'}
for colN in colL[:-1]:
    dataD[colN]=hsres.groupby(['state', ])[colN].sum()
    totalD[colN] = hsres[colN].sum()

dataD[colL[-1]] = hsres.groupby(['state', ])[colL[-1]].mean()
dataD['max percentage'] = hsres.groupby(['state', ])[colL[-1]].max()

totalD[colL[-1]] = hsres[colL[-1]].mean()
totalD['max percentage'] = dataD['max percentage'].mean()
df1=DataFrame(dataD)
df2=DataFrame(totalD, index=['last'])
df2=df2.set_index('state')
df = concat([df1,df2])
df.to_csv('state-summary.csv')
df


## Pick a state to focus in on

In [ ]:
# enter state abbreviation (e.g. "ny" or "NY" for displaying hospital results:
hstate = "ny" # @param {type:"string"}
hsres_hstate = hsres[hsres['state']==hstate.upper()]

fig,axL = plt.subplots(4,1, sharex=True, figsize=(12,9))
fig.suptitle(f'state={hstate.upper()}')
ylabL = ('#', '#', '#', 'fraction')
for stat, stat_avg, ax, ylab in zip( colL, col_avgL, axL, ylabL):
    this_state = hsres_hstate.groupby('date')[stat].sum()
    ax.plot_date(this_state.index,this_state,ms=1)
    this_state = hsres_hstate.groupby('date')[stat_avg].sum()
    ax.plot_date(this_state.index,this_state,'k-.', ms=1)
    ax.tick_params('x', rotation=90)
    ax.set_ylabel(ylab)
    ax.set_title(stat)
    format_axis(ax)

fig.savefig(f'{hstate.upper()}-adult_hosp_stats.png')

In [ ]:
# https://stackoverflow.com/questions/76593068/plotly-figure-not-rendering-in-ipywidgets-interact-function-google-colab
import plotly
plotly.io.renderers.default = 'colab'
import plotly.express as px
fig = px.bar(x=["a", "b", "c"], y=[1, 3, 2])

import plotly.graph_objects as go
fig_widget = go.FigureWidget(fig)
fig_widget